In [1]:
# Run me to hide code cells
from IPython.display import display, HTML
# from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = 
document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show inputs</button>"""))

In [2]:
# command line 
# conda install ipywidgets 
# jupyter nbextension enable --py widgetsnbextension


import os
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
from glob import glob
import random
from IPython.display import display
from ipywidgets import interact
import ipywidgets as widgets

import skimage
import pathlib

import src.ImagePreprocessFilters as IPrep
import src.ImageParser as IP
import src.jupyter_functions as JN

def set_params(res=300):
    plt.rcParams['figure.dpi'] = res
    plt.rcParams['savefig.dpi'] = res

cmap = plt.cm.gray   # for black and white
set_params(res=300)
# %matplotlib notebook
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
# import matplotlib
# matplotlib.use('nbagg')
# import matplotlib.pyplot as plt


In [3]:

# defining the Jupyter Widget 
def widget(result, channel_names):

    # helping functions
    def plot_percentile(images, COL_PLOT=2, RES=300):
        set_params(res=RES)
        if COL_PLOT>1:
            JN.plot_one_channel_side_by_side(images, columns=COL_PLOT, figsize=(70,70), cmap = cmap)

        else:
            JN.plot_one_channel(images, cmap = cmap) 

    def plot_comparison(images_or, imgs_filtered, res_compare, high_contrast):
        set_params(res=res_compare)
        JN.plot_compare_images(images_or, imgs_filtered, cmap = cmap, high_contrast = high_contrast)
#         JN.plot_compare_images_plotly(images_or, imgs_filtered, cmap = cmap, high_contrast = high_contrast)
    
    def plot_comparison_zoom(images_or, imgs_filtered, res_compare, high_contrast):
        output.clear_output()
        set_params(res=res_compare)
        JN.plot_compare_images_plotly(images_or, imgs_filtered, cmap = cmap, high_contrast = high_contrast)
    
    
    def do_and_plot_percentile(CH, PERCENTILE, TH, cols, res, sample_images,sample_images_number, high_contrast):
        output.clear_output()
        plot_output_histogram.clear_output()
        plot_output_images.clear_output()  
        plot_output_comparison.clear_output()  
        plot_output_psnr.clear_output()  

        files_channel = [file for file in result if str(dropdown_ch_name.value+'.ome.tiff') in str(file)]
        
        imgs_channel,norm_imgs_channel,imgs_filtered = JN.calculus_fun(files_channel, PERCENTILE, TH, 
                                                                    sample_images,sample_images_number)

        with plot_output_histogram:
            set_params(res=50)
            JN.histogram_one_channel_all_img(norm_imgs_channel)
            JN.histogram_one_channel_all_img(imgs_filtered)

        with plot_output_images:
            plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value)

        with plot_output_comparison:
            plot_comparison(imgs_channel, imgs_filtered, dropdown_res.value, 
                            high_contrast = dropdown_high_constrast_compare.value)
            
        with plot_output_ZOOM:
            plot_comparison_zoom(imgs_channel, imgs_filtered, dropdown_res.value, 
                            high_contrast = dropdown_high_constrast_compare.value)

#         with plot_output_psnr:
#             plot_psnr(imgs_channel,norm_imgs_channel, imgs_filtered)

        return imgs_channel,norm_imgs_channel,imgs_filtered

    def save_imgs(path_for_results, binary_masks = False):
        if not os.path.exists(path_for_results):
            os.makedirs(path_for_results)

        # names of images
        files_channel = [file for file in result if str(dropdown_ch_name.value+'.ome.tiff') in str(file)]

        paths_save = [str(path_for_results + os.path.basename(os.path.dirname(sub))) for sub in files_channel]

        names_save = [str(path_for_results + os.path.basename(os.path.dirname(sub)) + '/' + os.path.basename(sub)) 
                          for sub in files_channel]
        for path in paths_save:
            if not os.path.exists(path):
                os.makedirs(path)

        imgs_channel,norm_imgs_channel,imgs_filtered = JN.calculus_fun(
            files_channel, PERCENTILE = dropdown_percentile.value, TH = dropdown_th.value, 
            sample_images = 'all',sample_images_number = dropdown_sample_images_number.value)
        
        if binary_masks:
            imgs_filtered = [np.where(a > 0, 1, 0) for a in imgs_filtered]
            
        images_test = map(lambda p, f: IPrep.save_images(p, f, ch_last=True), imgs_filtered, names_save)
        it = list(images_test)
        print('Images saved at ',path_for_results)


    # dropdown functions 
    def dropdown_ch_name_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(change.new,dropdown_percentile.value,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)
    
    def dropdown_th_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value, dropdown_percentile.value, change.new,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)
    
    def dropdown_percentile_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value,change.new,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)   

    def dropdown_cols_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
            change.new, dropdown_res.value,dropdown_sample_images.value,dropdown_sample_images_number.value,
        dropdown_high_constrast_compare.value)

    def dropdown_res_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, change.new,dropdown_sample_images.value, dropdown_sample_images_number.value,
            dropdown_high_constrast_compare.value)

    def dropdown_sample_images_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,change.new,dropdown_sample_images_number.value,
            dropdown_high_constrast_compare.value)

    def dropdown_sample_images_number_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,change.new, dropdown_high_constrast_compare.value)
            
    def dropdown_high_constrast_compare_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                dropdown_sample_images_number.value,change.new)
            
    # events for saving     
    def btn_eventhandler(obj):
        print('Hello from the {} button!'.format(obj.description))
        print('Your images are being saved -np32 format ')

        path_for_results = txtbox.value
        save_imgs(path_for_results)
    
    def btn_bin_eventhandler(obj):
        print('Hello from the {} button!'.format(obj.description))
        print('Your Binary images are being saved - np32 format')
        path_for_results = txtbox.value
        save_imgs(path_for_results, binary_masks =True)      
        
    def txtbox_eventhandler(obj):
        path_for_results = obj.description
        
    # textbox 
    txtbox = widgets.Text(value='ResultsPercentile/', placeholder='Type something',
        description='PathSave:',disabled=False)

            
    # create outputs
    output = widgets.Output()
    plot_output_histogram = widgets.Output()
    plot_output_images = widgets.Output()
    plot_output_comparison = widgets.Output()
    plot_output_ZOOM = widgets.Output()

    plot_output_psnr = widgets.Output()

    # clear outputs
    plot_output_histogram.clear_output()
    plot_output_images.clear_output()
    plot_output_comparison.clear_output()
    plot_output_ZOOM.clear_output()

    plot_output_psnr.clear_output()

    # define dropdown plots images
    dropdown_cols = widgets.Dropdown(options = [1,2,3,4,5], description='Columns to display:')
    dropdown_res = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
    dropdown_sample_images = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], 
                                              description='Display a subset of images:')
    dropdown_sample_images_number = widgets.Dropdown(options = [1,2,3,4,5,6,7,8,9,10], description='Display a subset of images:')
    dropdown_high_constrast_compare = widgets.Dropdown(options = ['False', 'True'], description='Display on high constrast:')
    # define dropdown Denoise Parameters
    dropdown_ch_name = widgets.Dropdown(options = channel_names, description='Channel:')
    dropdown_th = widgets.Dropdown(options = [None,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9, 
                                              'otsu', 'isodata', 'Li', 'Yen', 'triangle', 'mean', 'local'
                                             ], description='threshold:')
    dropdown_percentile = widgets.Dropdown(options = [None,10,20,25,30,40,50,60,70,75,80,90,5,95,'p50consecutive'], description='Percentile:')


    dropdown_ch_name.observe(dropdown_ch_name_eventhandler, names='value')
    dropdown_th.observe(dropdown_th_eventhandler, names='value')
    dropdown_percentile.observe(dropdown_percentile_eventhandler, names='value')

    dropdown_cols.observe(dropdown_cols_eventhandler, names='value')
    dropdown_res.observe(dropdown_res_eventhandler, names='value')
    dropdown_sample_images.observe(dropdown_sample_images_eventhandler, names='value')
    dropdown_sample_images_number.observe(dropdown_sample_images_number_eventhandler, names='value')
    dropdown_high_constrast_compare.observe(dropdown_high_constrast_compare_eventhandler, names='value')
    
    # Dashboard
    input_widgets = widgets.HBox([dropdown_ch_name, dropdown_percentile,dropdown_th])
    input_widgetsimages = widgets.HBox([dropdown_sample_images_number, dropdown_sample_images])
    input_widgetsimages2 = widgets.HBox([dropdown_res,dropdown_cols, dropdown_high_constrast_compare])

    # create a container for the output. Tab is great for this. The 1st tab will host the dataframe and the 2nd one the graph
    tab = widgets.Tab([plot_output_images, plot_output_histogram, plot_output_comparison,plot_output_ZOOM, plot_output_psnr])
    tab.set_title(0, 'Images')
    tab.set_title(1, 'Histogram')
    tab.set_title(2, 'Compare images')
    tab.set_title(3, 'Compare ZOOM')
    tab.set_title(4, 'PSNR')

    # display(tab)
    # Define the layout for the widget container
    dashboard_layout = widgets.Layout(height='auto', overflow='auto',flex='1', max_height='100000px')

    # Create the dashboard with the custom layout     # Adjust the layout of the widget container to expand vertically

    dashboard = widgets.VBox([input_widgets, input_widgetsimages, input_widgetsimages2, tab], layout=dashboard_layout)
    
#     dashboard = widgets.VBox([input_widgets, input_widgetsimages,input_widgetsimages2,tab])
    display(dashboard)
    
    
    ## save part 
    btn = widgets.Button(description='Save')
    btn_bin = widgets.Button(description='Save Binary')
    save_btn = widgets.HBox([btn, btn_bin])
    display(save_btn)
    display(txtbox)
    btn.on_click(btn_eventhandler)
    btn_bin.on_click(btn_bin_eventhandler)



# IMAGES PER CHANNEL - One file correspondens to one channel image 

Folder structure:

-- Folder
   
   -- Folder_Pat_1
       
       -- img_ch_name1
       
       -- img_ch_name2
       
       -- img_ch_nameN
    
    -- Folder_Pat_2
    
        -- img_ch_name1
       
        -- img_ch_name2
       
        -- img_ch_nameN
    ...

    
The channel is a name of the channel  


In [4]:
from IPython.display import display, HTML
display(HTML("<style>div.output_scroll { height: 200em; }</style>"))
from IPython.display import clear_output

PATH = widgets.Text(value='data/IMC_ESD/raw', description='Path:', disabled=False)
button = widgets.Button(description="Change Path")
output_widget = widgets.Output()

def update_output(button):
    with output_widget:
        clear_output(wait=True)
        path = PATH.value
        if path:
            print("Path provided:", path)
            result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.ome.tiff'))]
            channel_names = set([name.split("_")[-1].split(".ome.tiff")[0] for name in result])
            widget(result, channel_names)

button.on_click(update_output)

display(PATH)
display(button)
display(output_widget)


Text(value='data/IMC_ESD/raw', description='Path:')

Button(description='Change Path', style=ButtonStyle())

Output()